# Generate Random Genome Sequences

This notebook generates `N` random of the length `K` with the minimal distance `MIN_DIST` to any Ensembl gene.

Note that some sequences may be missing in the human genome (= consists of 'N' letters only).

## Setup

Installation for colab environment.

In [1]:
!pip install biopython pyensembl

     |████████████████████████████████| 2.3MB 3.4MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 92kB 7.7MB/s 
  Created wheel for pyensembl: filename=pyensembl-1.8.7-cp36-none-any.whl size=56339 sha256=958342c70a49b5d5e02425b330ce7d0b49baa49d59d92f588b15b364b70e876d
  Stored in directory: /root/.cache/pip/wheels/69/8b/4d/a6c76aa39aa3619c8b8c5ff3e09cd180df57e5e9f04c655794
  Created wheel for typechecks: filename=typechecks-0.1.0-cp36-none-any.whl size=2770 sha256=63a9b1e7292beaf2311dfa5b9c4421fa9e0e7cd1fe821a782974987917298ab3
  Stored in directory: /root/.cache/pip/wheels/27/ad/82/4a9bbc88f1d31bb1a0c053e29efafc02f56e2ed1598f5639b2
  Created wheel for datacache: filename=datacache-1.1.5-cp36-none-any.whl size=16472 sha256=84724c7b783ef7cfa567b13b47f736a4564ba60185161e9d739128f09b8c6234
  Stored in directory: /root/.cache/pip/wheels/6b/89/37/76d23043c61b5096dd0170fd6ea2f6474b38d69965b633118a
  Created wheel for memoized-property: filen

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!pyensembl install --release 97 --species human

2020-06-23 21:43:21,757 - pyensembl.shell - INFO - Running 'install' for EnsemblRelease(release=97, species='homo_sapiens')
2020-06-23 21:43:21,757 - pyensembl.download_cache - INFO - Fetching /root/.cache/pyensembl/GRCh38/ensembl97/Homo_sapiens.GRCh38.97.gtf.gz from URL ftp://ftp.ensembl.org/pub/release-97/gtf/homo_sapiens/Homo_sapiens.GRCh38.97.gtf.gz
2020-06-23 21:43:21,757 - datacache.download - INFO - Downloading ftp://ftp.ensembl.org/pub/release-97/gtf/homo_sapiens/Homo_sapiens.GRCh38.97.gtf.gz to /root/.cache/pyensembl/GRCh38/ensembl97/Homo_sapiens.GRCh38.97.gtf.gz
2020-06-23 21:43:25,149 - pyensembl.download_cache - INFO - Fetching /root/.cache/pyensembl/GRCh38/ensembl97/Homo_sapiens.GRCh38.cdna.all.fa.gz from URL ftp://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz
2020-06-23 21:43:25,150 - datacache.download - INFO - Downloading ftp://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz to /root/.

In [4]:
import pandas as pd
import numpy as np
import gzip
from tqdm.notebook import tqdm

from Bio import SeqIO   # for reading fasta files
from pyensembl import EnsemblRelease   # to get the gene list

ENSEMBL_RELEASE = 97
DNA_TOPLEVEL_FASTA_PATH = "/content/drive/My Drive/data/ensembl/Homo_sapiens.GRCh38.dna.toplevel.fa.gz"

# to generate random sequences
N = 50_000    # how many
K = 200       # how long
MIN_DIST = 50 # how far should they be from the gene
OUTPUT_FILE = '/content/drive/My Drive/data/random/random_seqs3.csv'   # where to save them
# OUTPUT_FILE = 'random_seqs.csv'

CHRS = [str(chr) for chr in range(1,23)] + ['X', 'Y', 'MT']

## Get length of each chromosome

In [5]:
def get_chr_lengths(fasta_path=DNA_TOPLEVEL_FASTA_PATH):
    chr_lengths = {}
    
    with gzip.open(fasta_path, "rt") as handle:
        for record in tqdm(SeqIO.parse(handle, "fasta"), total=24):
            chr_lengths[record.id] = len(record.seq)
            if record.id == "MT": 
                # stop, do not read small contigs
                break
    
    # check that we have all chromosomes
    assert set(chr_lengths.keys()) == set(CHRS) 
    
    return chr_lengths

contig_lengths = get_chr_lengths()
contig_lengths

{'1': 248956422,
 '10': 133797422,
 '11': 135086622,
 '12': 133275309,
 '13': 114364328,
 '14': 107043718,
 '15': 101991189,
 '16': 90338345,
 '17': 83257441,
 '18': 80373285,
 '19': 58617616,
 '2': 242193529,
 '20': 64444167,
 '21': 46709983,
 '22': 50818468,
 '3': 198295559,
 '4': 190214555,
 '5': 181538259,
 '6': 170805979,
 '7': 159345973,
 '8': 145138636,
 '9': 138394717,
 'MT': 16569,
 'X': 156040895,
 'Y': 57227415}

In [6]:
total_length = pd.Series(contig_lengths).sum()
total_length

3088286401

## Get gene list

In [7]:
# release 97 uses human reference genome GRCh38
data = EnsemblRelease(ENSEMBL_RELEASE)

In [8]:
human_genes = data.genes()
len(human_genes)

60617

In [9]:
human_genes[0]

Gene(gene_id='ENSG00000000003', gene_name='TSPAN6', biotype='protein_coding', contig='X', start=100627109, end=100639991, strand='-', genome='GRCh38')

In [10]:
human_genes_tuples = [(x.gene_id, x.gene_name, x.biotype, x.contig, x.start, x.end, x.strand) for x in human_genes]
human_genes_table = pd.DataFrame.from_records(human_genes_tuples, columns=["id", "symbol", "biotype", "chr", "start", "end", "strand"])
assert all(human_genes_table.start <= human_genes_table.end)

human_genes_table.head()

,id,symbol,biotype,chr,start,end,strand
0,ENSG00000000003,TSPAN6,protein_coding,X,100627109,100639991,-
1,ENSG00000000005,TNMD,protein_coding,X,100584936,100599885,+
2,ENSG00000000419,DPM1,protein_coding,20,50934867,50958555,-
3,ENSG00000000457,SCYL3,protein_coding,1,169849631,169894267,-
4,ENSG00000000460,C1orf112,protein_coding,1,169662007,169854080,+


In [11]:
excluded_regions = human_genes_table[["chr", "start", "end"]][human_genes_table.chr.isin(CHRS)].copy()
excluded_regions.chr.value_counts()

1     5471
2     4196
11    3360
3     3185
17    3060
6     3059
12    3054
7     3014
19    2992
5     2983
4     2651
16    2556
8     2482
X     2422
10    2332
9     2327
14    2282
15    2221
20    1457
13    1397
22    1384
18    1242
21     872
Y      522
MT      37
Name: chr, dtype: int64

In [12]:
excluded_regions.head()

,chr,start,end
0,X,100627109,100639991
1,X,100584936,100599885
2,20,50934867,50958555
3,1,169849631,169894267
4,1,169662007,169854080


In [13]:
total_gene_length = ((excluded_regions.end - excluded_regions.start) + 1).sum()
total_gene_length, total_gene_length/total_length

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


(1976405758, 0.6399684165820992)

In [14]:
excluded_regions['start_dist'] = excluded_regions['start'] - MIN_DIST - K
excluded_regions['end_dist'] = excluded_regions['end'] + MIN_DIST

## Check on chr1 how many possible seqs are really there

In [15]:
c = '1'
c_len = contig_lengths[c]
possible_seq_starts = np.ones(c_len, dtype=bool)

for i in tqdm(range(excluded_regions.shape[0])):
    if excluded_regions.chr.values[i] == c:
        start = max(excluded_regions.start_dist.values[i], 1)
        end = min(excluded_regions.end_dist.values[i], c_len)
        possible_seq_starts[(start-1):end] = 0

In [16]:
possible_seq_starts.sum(), possible_seq_starts.sum()/c_len

(101541431, 0.4078682935120268)

In [17]:
biotypes = pd.Series([g.biotype for g in human_genes])
biotypes.value_counts()

protein_coding                        19986
lncRNA                                16828
processed_pseudogene                  10170
unprocessed_pseudogene                 2626
misc_RNA                               2220
snRNA                                  1910
miRNA                                  1879
TEC                                    1064
snoRNA                                  942
transcribed_unprocessed_pseudogene      916
rRNA_pseudogene                         499
transcribed_processed_pseudogene        491
IG_V_pseudogene                         188
IG_V_gene                               144
transcribed_unitary_pseudogene          129
TR_V_gene                               106
unitary_pseudogene                       97
TR_J_gene                                79
rRNA                                     58
scaRNA                                   49
polymorphic_pseudogene                   42
IG_D_gene                                37
TR_V_pseudogene                 

## Random sequence generation

In [18]:
def get_random_chr(chr_lengths: pd.Series):
    chr_probs = chr_lengths / chr_lengths.sum()
    return CHRS[np.argwhere(np.random.multinomial(1, chr_probs))[0][0]]

def is_intersecting(c, pos, df_forbidden):
    intersecting = (df_forbidden.chr.values == c) & (df_forbidden.start_dist.values <= pos) & (df_forbidden.end_dist.values >= pos)
    return intersecting.any()

def get_random_pos(df_forbidden: pd.DataFrame, chr_lengths: pd.Series):
    c = get_random_chr(chr_lengths)
    c_len = chr_lengths[c]
    pos = np.random.randint(c_len) + 1
    
    while is_intersecting(c, pos, df_forbidden):
        pos = np.random.randint(c_len) + 1
    
    return c, pos

In [19]:
chr_lengths = pd.Series(contig_lengths)
get_random_chr(chr_lengths) 

'17'

In [20]:
is_intersecting('1', 11_000_000, excluded_regions)

False

In [21]:
get_random_pos(excluded_regions, chr_lengths)

('14', 39388941)

In [22]:
random_seqs = [get_random_pos(excluded_regions, chr_lengths) for i in tqdm(range(N))]

## Get actual genomic sequences

In [23]:
seqs = pd.DataFrame.from_records(random_seqs, columns=["chr", "start"])
seqs['end'] = seqs['start'] + K - 1
seqs['seq'] = ''

In [24]:
seqs.head()

,chr,start,end,seq
0,5,123461076,123461275,
1,5,52230676,52230875,
2,5,71940500,71940699,
3,4,69974223,69974422,
4,13,13243412,13243611,


In [25]:
def which(self):
    try:
        self = list(iter(self))
    except TypeError as e:
        raise Exception("""'which' method can only be applied to iterables.
        {}""".format(str(e)))
    indices = [i for i, x in enumerate(self) if bool(x) == True]
    return(indices)

with gzip.open(DNA_TOPLEVEL_FASTA_PATH, "rt") as handle:
    for record in tqdm(SeqIO.parse(handle, "fasta"), total=24):
        sel_seqs = which(seqs.chr == record.id)
        for i in sel_seqs:
            seqs.loc[i, "seq"] = str(record.seq[(seqs.start[i]-1):seqs.end[i]])
        
        if record.id == "MT": 
            # stop, do not read small contigs
            break

In [26]:
seqs.head()

,chr,start,end,seq
0,5,123461076,123461275,CCTATGACTAGTCTATTTTTTTTTTTTTTTTTTTTTTGAGACAGAG...
1,5,52230676,52230875,ATTATTCAGTGATTGAATCTGTGCTAACGAAGCTGCTTTGTTCACG...
2,5,71940500,71940699,AACACATATAATTACCAGGAATGCAAAAAAAAAGAAAACATAAATA...
3,4,69974223,69974422,TGGATGCCTTATATTTCTTTTGTCTGCTTGCTCTGGATATGACTAC...
4,13,13243412,13243611,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...


In [27]:
len(seqs.seq.values[0])

200

In [28]:
seqs.seq.values[1]

'ATTATTCAGTGATTGAATCTGTGCTAACGAAGCTGCTTTGTTCACGGAGGTGTCATCTGGAAGGTGAGCCTGCCCTTGTTCTGGGAGGGTATTGAATGGGTGTTGTTTTCCTCTTGTTTTTGTGGGACAAGATTCTCTTTCACAACCTCAGGGAATCTACAAACATTCTAGTAACTTCTTTTTCCCCCCACTTACTTTGA'

## Save generated sequences to file

In [29]:
seqs.to_csv(OUTPUT_FILE, index=False)